End goal:
accessibility table by MTC zone
by mode, 15 / 30 / 45 min

Process:
    1. MTC TAZ shapefile
    2. Load skims
    3. Get number of jobs within each zone
    4. For each zone, perform accessibility calculations by mode and time limit

In [81]:
import os; os.chdir('..')
import pandas as pd
import numpy as np
import orca
from scripts import datasources, models, variables

import pyproj as proj
from shapely.geometry import Point
from shapely import geometry
import geopandas as gpd


In [5]:
@orca.table(cache=False)
def nodessmall():
    df = pd.read_csv('/home/data/2018-07/nodessmall_vars.csv')
    df.rename(columns = {'osmid':'node_id_small'}, inplace = True)
    df.set_index('node_id_small',inplace = True)
    return df

@orca.table(cache=False)
def nodeswalk():
    df = pd.read_csv('/home/data/2018-07/nodeswalk_vars.csv')
    df.rename(columns = {'osmid':'node_id_walk'}, inplace = True)
    df.set_index('node_id_walk',inplace = True)
    return df

#initialization only
orca.run(['initialize_network_small','initialize_network_walk'])

Running step 'initialize_network_small'
Time to execute step 'initialize_network_small': 0.00 s
Running step 'initialize_network_walk'
Time to execute step 'initialize_network_walk': 0.00 s
Total time to execute iteration 1 with iteration value None: 0.00 s


In [63]:
df['job_count'] = df.groupby('node_id').transform('count')['job_id']
df = df[['node_id','x','y','job_count']].drop_duplicates('node_id')

In [ ]:
df = orca.merge_tables(target = 'jobs',tables = ['jobs','buildings','parcels'])
print(len(df))
df = df.reset_index()[['job_id','node_id','x','y']]
df['job_count'] = df.groupby('node_id').transform('count')['job_id']
df = df[['node_id','x','y','job_count']].drop_duplicates('node_id')

In [66]:
TAZ = gpd.read_file('/home/jayne/ual_model_workspace/fall-2018-models/notebooks-jayne/mtc_TAZ/Transportation_Analysis_Zones.shp')


from geopandas import GeoDataFrame
from shapely.geometry import Point

geometry = [Point(xy) for xy in zip(df.x, df.y)]
df = df.drop(['x', 'y'], axis=1)
crs = {'init': 'epsg:4326'}
parcel_gdf = GeoDataFrame(df, crs=crs, geometry=geometry)


In [67]:
parcel_with_TAZ = gpd.sjoin(parcel_gdf, TAZ, how="left", op='within')
TAZ_jobs = parcel_with_TAZ.groupby('taz1454').sum()['job_count'].reset_index()

In [ ]:
skim = pd.read_csv('/home/data/fall_2018/mtc_skims/TimeSkimsDatabaseMD.csv')
skim.replace(-999,999,inplace = True) # negative impedance to positive impedance

In [41]:
def jobs_by_mode(orig_zone, mode, tt_limit):
    travel_times = skim[skim.orig == orig_zone][[mode,'dest']]
    return travel_times.query(f'{mode}<={tt_limit}').merge(TAZ_jobs,
            how = 'left', left_on = 'dest',right_on = 'taz1454').job_count.sum()

In [76]:
modes = ['da','walk','bike','wTrnW']
for mode in modes:
    for tt_limit in [15,30,45]:
        TAZ_jobs[f'{mode}_{tt_limit}'] = TAZ_jobs.apply(lambda row: jobs_by_mode(row.taz1454, mode, tt_limit), axis = 1)

In [78]:
TAZ_jobs.to_csv('/home/jayne/ual_model_workspace/fall-2018-models/data/MTC_TAZ_accessibility.csv',index = False)